<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/Facebook_Data_Scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install selenium bs4 webdriver-manager pandas

In [8]:
!pip install chromedriver-autoinstaller

In [5]:
import os

# On Linux/macOS:
os.environ['FB_EMAIL'] = "chote@jbnote.com"
os.environ['FB_PASSWORD'] = "Rm(bb7SBuyj6q7s"

# On Windows (PowerShell):
# os.environ['FB_EMAIL'] = "chote@jbnote.com"
# os.environ['FB_PASSWORD'] = "Rm(bb7SBuyj6q7s"

In [11]:
!apt-get update
!apt-get install google-chrome-stable

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,152 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [13]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f # Install dependencies

--2025-07-28 06:03:25--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.141.93, 142.250.141.136, 142.250.141.190, ...
Connecting to dl.google.com (dl.google.com)|142.250.141.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118146996 (113M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 112.67M   291MB/s    in 0.4s    

2025-07-28 06:03:27 (291 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [118146996/118146996]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (138.0.7204.168-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Packa

In [1]:
pip install selenium webdriver-manager facebook-scraper pandas

In [3]:
#!/usr/bin/env python3
"""
scrape_ghorerbazarbd_requests.py

1) Selenium logs into www.facebook.com (desktop) → harvests cookies
2) requests.Session() uses those cookies on mbasic.facebook.com
3) Follows “Older Posts” links to paginate
4) Stops after 400 posts or when no more pages
5) Saves ID, URL, timestamp, text, likes, comments, shares → CSV
"""

import time
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ─── CONFIG ────────────────────────────────────────────────────────────────────

FB_EMAIL    = "chote@jbnote.com"
FB_PASSWORD = "Rm(bb7SBuyj6q7s"
PAGE_HANDLE = "Ghorerbazarbd.comm"
POST_LIMIT  = 400
OUTPUT_CSV  = "ghorerbazarbd_posts.csv"

# ─── 1) Use Selenium to log in and grab cookies ────────────────────────────────

def fetch_facebook_cookies():
    opts = Options()
    # You can remove headless if Facebook blocks it:
    opts.add_argument("--headless")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=opts
    )
    try:
        driver.get("https://www.facebook.com/login")
        time.sleep(2)
        driver.find_element(By.NAME, "email").send_keys(FB_EMAIL)
        driver.find_element(By.NAME, "pass").send_keys(FB_PASSWORD)
        driver.find_element(By.NAME, "login").click()
        time.sleep(5)  # let post‑login redirect finish

        # extract cookies into a dict
        ck = {}
        for c in driver.get_cookies():
            ck[c["name"]] = c["value"]

        # sanity check
        if "c_user" not in ck or "xs" not in ck:
            raise RuntimeError("Login didn’t yield c_user/xs cookies; check credentials or 2FA.")
        return ck

    finally:
        driver.quit()

# ─── 2) Use requests + those cookies to scrape mbasic.facebook.com ────────────

def scrape_with_requests(cookies):
    session = requests.Session()
    # inject selenium cookies into requests
    jar = requests.cookies.RequestsCookieJar()
    for k, v in cookies.items():
        jar.set(k, v, domain=".facebook.com", path="/")
    session.cookies.update(jar)

    url = f"https://mbasic.facebook.com/{PAGE_HANDLE}"
    posts = []
    seen = set()

    while url and len(posts) < POST_LIMIT:
        resp = session.get(url)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # 1) Extract all posts on this page
        for art in soup.find_all("div", {"role": "article"}):
            # get link → ID & URL
            a = art.find("a", href=True)
            if not a:
                continue
            href = a["href"]
            post_url = "https://mbasic.facebook.com" + href
            if "story_fbid=" in href:
                pid = href.split("story_fbid=")[1].split("&")[0]
            else:
                pid = href

            if pid in seen:
                continue
            seen.add(pid)

            # text: join stripped strings, filter out “See more”
            txt = " ".join(
                t for t in art.stripped_strings
                if t.lower() not in ("see more",)
            )

            # timestamp
            abbr = art.find("abbr")
            ts = abbr["data-utime"] if (abbr and abbr.has_attr("data-utime")) else ""

            # counts (likes/comments/shares) in the next <div>
            likes = comments = shares = 0
            foot = art.find_next_sibling("div")
            if foot:
                for link in foot.find_all("a"):
                    parts = link.get_text(strip=True).split()
                    if len(parts) >= 2 and parts[0].isdigit():
                        cnt = int(parts[0])
                        if "Like" in parts[1]:
                            likes = cnt
                        elif "Comment" in parts[1]:
                            comments = cnt
                        elif "Share" in parts[1]:
                            shares = cnt

            posts.append({
                "post_id":  pid,
                "post_url": post_url,
                "timestamp": ts,
                "text":     txt,
                "likes":    likes,
                "comments": comments,
                "shares":   shares
            })
            print(f"[{len(posts):3d}/{POST_LIMIT}] post {pid}")

            if len(posts) >= POST_LIMIT:
                break

        if len(posts) >= POST_LIMIT:
            break

        # 2) Find “Older Posts” link to paginate
        older = soup.find("a", string=lambda s: s and "older posts" in s.lower())
        if older and older.has_attr("href"):
            url = "https://mbasic.facebook.com" + older["href"]
            time.sleep(1)  # be kind
        else:
            print("↳ no more “Older Posts” link found.")
            break

    return posts

# ─── 3) Save to CSV ────────────────────────────────────────────────────────────

def save_to_csv(posts):
    keys = ["post_id","post_url","timestamp","text","likes","comments","shares"]
    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(posts)
    print(f"\n✅ Saved {len(posts)} posts → {OUTPUT_CSV}")

# ─── MAIN ───────────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    print("🔐 Logging into Facebook via Selenium…")
    ck = fetch_facebook_cookies()
    print("🚀 Scraping via requests & mbasic.facebook.com…")
    data = scrape_with_requests(ck)
    if data:
        save_to_csv(data)
    else:
        print("⚠️  No posts found—check that your account can view that page.")

🔐 Logging into Facebook via Selenium…


RuntimeError: Login didn’t yield c_user/xs cookies; check credentials or 2FA.